In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import mne
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import time
import logging
import warnings
import pandas as pd
from tqdm import tqdm

In [3]:
logging.disable(logging.CRITICAL)
warnings.filterwarnings("ignore")

# Load Data

In [4]:
def get_subject_id(file_path):
    parts = file_path.split(os.sep)
    subject = parts[-2]      # e.g., "S003"
    subject_num = int(subject[1:])  # converts "003" to integer 3
    return subject_num

def load_eeg_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload = True, verbose = False)
    raw.pick(['Oz..', 'T7..', 'Cz..'])    
    raw.filter(1., 40., fir_design = 'firwin', verbose = False)
    # raw.crop(tmin=0, tmax=30) #remove cropping
    return raw

eeg_file_path = "./files/S003/S003R03.edf"
raw = load_eeg_data(eeg_file_path)
print(raw.annotations)


<Annotations | 30 segments: T0 (15), T1 (7), T2 (8)>


## Generate Dataset

In [5]:
class EEGMotorImageryDataset(Dataset):
    def __init__(self, epochs, subject_id):
        data = epochs.get_data()
        valid_idx = [i for i, epoch in enumerate(data) if epoch.shape[-1] == 160]
    
        self.data = data[valid_idx]
        self.labels = np.full((self.data.shape[0],), subject_id) 

        self.data = (self.data - np.mean(self.data, axis = 2, keepdims = True)) / np.std(self.data, axis = 2, keepdims = True)
        self.data = self.data.astype(np.float32)
        self.indices_by_class = {subject_id: list(range(len(self.data)))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        sample = sample[np.newaxis,:,:]
        return sample, label



In [10]:
def load_file_as_dataset(file_path):

    raw = load_eeg_data(file_path)
    sfreq = raw.info['sfreq']
    expected_length = int(round(sfreq * 1.0))  # for 1 second epochs

    epochs = mne.make_fixed_length_epochs(raw, duration=1.0, preload=True, verbose=False)


    data = epochs.get_data() 
    valid_idx = [i for i, epoch in enumerate(data) if epoch.shape[-1] == expected_length]
    filtered_epochs = epochs[valid_idx]

    subject_id = get_subject_id(file_path)
    return EEGMotorImageryDataset(filtered_epochs, subject_id=subject_id)

In [11]:
def load_all_data(folder_path):
    edf_files = []
    for root, dir, files in os.walk(folder_path):
        for f in files:
            if f.lower().endswith('.edf'):
                edf_files.append(os.path.join(root, f))

    datasets = []
    for file_path in tqdm(edf_files):
        ds = load_file_as_dataset(file_path)
        datasets.append(ds)
    if datasets:
        combined_dataset = ConcatDataset(datasets)
        return combined_dataset
    else:
        return None

In [12]:
folder_path = "./files"
dataset = load_all_data(folder_path)

if dataset is None:
    print('No EDF files found')
else: 
    print("Total samples in combined_dataset:", len(dataset))

100%|██████████| 1526/1526 [01:14<00:00, 20.47it/s]

Total samples in combined_dataset: 170272


## Siamse Dataset

In [13]:
class SiameseEEGMotorImageryDataset(Dataset):
    def __init__(self, base_dataset):
        self.base_dataset = base_dataset
        self.indices_by_subject = {}
        for idx in range(len(self.base_dataset)):
            _, label = self.base_dataset[idx]
            if label not in self.indices_by_subject:
                self.indices_by_subject[label] = []
            self.indices_by_subject[label].append(idx)
        self.all_subjects = list(self.indices_by_subject.keys())


    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        sample1, label1 = self.base_dataset[idx]

        if np.random.rand() < 0.5:
            indices = self.indices_by_subject[label1]
            canidate = np.random.choice([i for i in indices if i != idx])
            sample2, _ = self.base_dataset[canidate]
            pair_label = 1
        else:
            other_subjects = [sub for sub in self.all_subjects if sub != label1]
            negative_label = np.random.choice(other_subjects)
            indices = self.indices_by_subject[negative_label]
            canidate = np.random.choice(indices)
            sample2, _ = self.base_dataset[canidate]
            pair_label = 0

        return (sample1, sample2), pair_label

siamese_dataset = SiameseEEGMotorImageryDataset(dataset)

## EEGNet

In [14]:
num_input = 1
F1 = 8
D = 2
F2 = 16

kernel_size_1 = (1, 64)  
kernel_padding_1 = (0, 32)
kernel_size_2 = (2, 32)  
kernel_avgpool_1 = (1, 8)
dropout_rate = 0.5
kernel_size_3 = (1, 16)  
kernel_padding_3 = (0, 8)
kernel_size_4 = (1, 1)   
kernel_avgpool_2 = (1, 4)
signal_length = 160      
embedding_dim = 128 

In [15]:
class EEGNetFeature(nn.Module): 
    def __init__(self):
        super(EEGNetFeature, self).__init__()
        # Layer 1
        self.conv2d = nn.Conv2d(num_input, F1, kernel_size=kernel_size_1, padding=kernel_padding_1)
        self.Batch_normalization_1 = nn.BatchNorm2d(F1)
        # Layer 2
        self.Depthwise_conv2D = nn.Conv2d(F1, D * F1, kernel_size=kernel_size_2, groups=F1)
        self.Batch_normalization_2 = nn.BatchNorm2d(D * F1)
        self.Elu = nn.ELU()
        self.Average_pooling2D_1 = nn.AvgPool2d(kernel_avgpool_1)
        self.Dropout = nn.Dropout2d(dropout_rate)
        # Layer 3
        self.Separable_conv2D_depth = nn.Conv2d(D * F1, D * F1, kernel_size=kernel_size_3,
                                                 padding=kernel_padding_3, groups=D * F1)
        self.Separable_conv2D_point = nn.Conv2d(D * F1, F2, kernel_size=kernel_size_4)
        self.Batch_normalization_3 = nn.BatchNorm2d(F2)
        self.Average_pooling2D_2 = nn.AvgPool2d(kernel_avgpool_2)
        # Layer 4
        self.Flatten = nn.Flatten()
        # Compute the flattened feature size. This depends on your input signal size.
        # Here we assume the pooling operations reduce the time dimension by a factor of 32.
        self.Dense = nn.Linear(128, embedding_dim)
        # Note: we remove the Softmax to get raw embeddings (or logits) for the siamese branch

    def forward(self, x):
        # Layer 1
        y = self.conv2d(x)
        y = self.Batch_normalization_1(y)
        # Layer 2
        y = self.Depthwise_conv2D(y)
        y = self.Batch_normalization_2(y)
        y = self.Elu(y)
        y = self.Average_pooling2D_1(y)
        y = self.Dropout(y)
        # Layer 3
        y = self.Separable_conv2D_depth(y)
        y = self.Separable_conv2D_point(y)
        y = self.Batch_normalization_3(y)
        y = self.Elu(y)
        y = self.Average_pooling2D_2(y)
        y = self.Dropout(y)
        # Layer 4
        y = self.Flatten(y)
        y = self.Dense(y)
        return y  # These are your embeddings (or logits)

In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(num_input, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(61440, 1024)
        self.dense2 = nn.Linear(1024, 128)
        self.dropout = nn.Dropout2d(0.5) #hyperparameter

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)

        x = F.relu(self.conv3(x))
        
        x = self.flatten(x)

        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.dense2(x)
        return x

## SiameseEEGNet

In [17]:
class SiameseEEGNet(nn.Module):
    def __init__(self):
        super(SiameseEEGNet, self).__init__()
        # Shared EEGNet feature extractor (weights will be shared for both inputs)
        self.feature_extractor = EEGNetFeature()
        
    def forward(self, x1, x2):
        # Get embeddings for both inputs
        embed1 = self.feature_extractor(x1)
        embed2 = self.feature_extractor(x2)
        return embed1, embed2


# Example usage:
# Create the Siamese model instance
siamese_model = SiameseEEGNet()

In [18]:
class SiameseCNN(nn.Module):
    def __init__(self):
        super(SiameseCNN, self).__init__()
        # Shared EEGNet feature extractor (weights will be shared for both inputs)
        self.feature_extractor = CNN()
        
    def forward(self, x1, x2):
        # Get embeddings for both inputs
        embed1 = self.feature_extractor(x1)
        embed2 = self.feature_extractor(x2)
        return embed1, embed2


# Example usage:
# Create the Siamese model instance
siamese_model2 = SiameseCNN()

### Loss Function

In [19]:
def contrastive_loss(y_true, cos_sim, margin = 1.0):
    loss_similar = y_true * torch.pow((1-cos_sim), 2)

    loss_dissimilar = (1-y_true) * torch.pow(torch.clamp(cos_sim - margin, min = 0.0), 2)
    loss = torch.mean(loss_similar + loss_dissimilar)
    return loss

### Training Loss and Validation

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Instantiate your model and move it to device.
siamese_model = SiameseCNN().to(device)

# Hyperparameters
batch_size = 32
learning_rate = 1e-3
num_epochs = 10
threshold = 0.725
# Split the dataset into training and validation (80%/20% split)
dataset_size = len(siamese_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = torch.utils.data.random_split(siamese_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer (using Adam here)
optimizer = torch.optim.Adam(siamese_model.parameters(), lr=learning_rate)

# Optionally, define a scheduler (e.g., ReduceLROnPlateau) if desired:
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# For tracking best validation loss to save the best model
best_val_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    siamese_model.train()  # set model to training mode
    running_loss = 0.0
    epoch_start_time = time.time()
    
    # Training step
    for batch_idx, ((x1, x2), labels) in enumerate(train_loader):
        # Move data to device
        x1 = x1.to(device)
        x2 = x2.to(device)
        labels = labels.to(device)  # Expected shape: (batch_size, 1)
        
        optimizer.zero_grad()
        
        # Forward pass: compute embeddings and cosine similarity
        embed1, embed2 = siamese_model(x1, x2)
        
        # Compute loss using cosine-based contrastive loss
        cos_sim = F.cosine_similarity(embed1, embed2, dim=1, eps=1e-6).unsqueeze(1)
        loss = contrastive_loss(labels, cos_sim, margin=0.9)
        
        # Backpropagation and optimization step
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * x1.size(0)
    
    # Compute average training loss for the epoch
    train_loss = running_loss / len(train_loader.dataset)
    
    # Validation step (without gradient computations)
    siamese_model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for (x1, x2), labels in val_loader:
            x1 = x1.to(device)
            x2 = x2.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            
            # Forward pass on validation data
            embed1, embed2 = siamese_model(x1, x2)
            cos_sim = F.cosine_similarity(embed1, embed2, dim=1, eps=1e-6).unsqueeze(1)
            loss = contrastive_loss(labels, cos_sim, margin=0.9)
            val_running_loss += loss.item() * x1.size(0)

            preds = (cos_sim > threshold).float()
            correct += (preds == labels).float().sum().item()
            total += labels.size(0)
    
    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    
    epoch_duration = time.time() - epoch_start_time
    print(f"Epoch {epoch+1}/{num_epochs} | Time: {epoch_duration:.2f}s | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    
    # Optionally, step the scheduler:
    # scheduler.step(val_loss)
    
    # Save the model if validation loss decreases
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(siamese_model.state_dict(), 'best_siamese_model.pth')
        print("  --> Best model saved.")

print("Training complete.")


Using device: cpu


KeyboardInterrupt: 